In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [23]:
df = pd.read_csv('flight.csv')
df = df.drop(['tailnum','dep_time','sched_dep_time','dep_delay'],axis=1)
print('Data Types:')
print(df.dtypes.value_counts())
print()
print(df.info())
df.head()

Data Types:
object     4
int64      3
float64    2
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 9 columns):
arr_time          328063 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
origin            336776 non-null object
dest              336776 non-null object
distance          336776 non-null int64
time_hour         336776 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 23.1+ MB
None


,arr_time,sched_arr_time,arr_delay,carrier,flight,origin,dest,distance,time_hour
0,830.0,819,11.0,UA,1545,EWR,IAH,1400,2013-01-01 05:00:00
1,850.0,830,20.0,UA,1714,LGA,IAH,1416,2013-01-01 05:00:00
2,923.0,850,33.0,AA,1141,JFK,MIA,1089,2013-01-01 05:00:00
3,1004.0,1022,-18.0,B6,725,JFK,BQN,1576,2013-01-01 05:00:00
4,812.0,837,-25.0,DL,461,LGA,ATL,762,2013-01-01 06:00:00


In [24]:
df['time_hour'] = pd.to_datetime(df['time_hour'])
df['date'] = df['time_hour'].map(lambda x: x.strftime('%Y-%m-%d'))
df['distance(miles)'] = round(df.distance.apply(lambda x: x*0.62137),2)
df.head()

,arr_time,sched_arr_time,arr_delay,carrier,flight,origin,dest,distance,time_hour,date,distance(miles)
0,830.0,819,11.0,UA,1545,EWR,IAH,1400,2013-01-01 05:00:00,2013-01-01,869.92
1,850.0,830,20.0,UA,1714,LGA,IAH,1416,2013-01-01 05:00:00,2013-01-01,879.86
2,923.0,850,33.0,AA,1141,JFK,MIA,1089,2013-01-01 05:00:00,2013-01-01,676.67
3,1004.0,1022,-18.0,B6,725,JFK,BQN,1576,2013-01-01 05:00:00,2013-01-01,979.28
4,812.0,837,-25.0,DL,461,LGA,ATL,762,2013-01-01 06:00:00,2013-01-01,473.48


In [25]:
pd.concat([df.isnull().sum(), 100 * df.isnull().sum()/len(df)], 
          axis=1).rename(columns={0:'Missing Records',   1:'Percentage (%)'})

,Missing Records,Percentage (%)
arr_time,8713,2.587180
sched_arr_time,0,0.000000
arr_delay,9430,2.800081
carrier,0,0.000000
flight,0,0.000000
origin,0,0.000000
dest,0,0.000000
distance,0,0.000000
time_hour,0,0.000000
date,0,0.000000


In [26]:
cols = ['dep_time', 'dep_delay', 'arr_time', 'arr_delay']
df[cols] = df[cols].fillna(df[cols].median())
df.head()

KeyError: "['dep_time' 'dep_delay'] not in index"

In [27]:
df['month'] = df['time_hour'].dt.month
pd.crosstab(df['origin'], df['month'])

month,1,2,3,4,5,6,7,8,9,10,11,12
origin,,,,,,,,,,,,
EWR,9893,9107,10420,10531,10592,10175,10475,10359,9550,10104,9707,9922
JFK,9161,8421,9697,9218,9397,9472,10023,9983,8908,9143,8710,9146
LGA,7950,7423,8717,8581,8807,8596,8927,8985,9116,9642,8851,9067


In [28]:
pd.pivot_table(df, values='arr_delay', index=['origin'],
                  columns=['month'], aggfunc=np.mean)

month,1,2,3,4,5,6,7,8,9,10,11,12
origin,,,,,,,,,,,,
EWR,12.816556,8.775160,10.600799,14.143388,5.381928,16.863599,15.460201,6.712342,-4.729972,2.604737,0.672498,19.639745
JFK,1.368398,4.391033,2.580815,7.011539,2.122977,17.596929,20.190222,5.910841,-4.463018,-3.585972,-0.872874,12.677575
LGA,3.382402,3.147389,3.738498,12.038582,2.796376,14.769278,14.181570,5.407801,-2.825395,0.186423,1.551187,11.956372


In [29]:
df['sum_arr_delay'] = df.groupby('carrier')['arr_delay'].transform(pd.Series.cumsum)
df[df.carrier=='UA'].head(10)

,arr_time,sched_arr_time,arr_delay,carrier,flight,origin,dest,distance,time_hour,date,distance(miles),month,sum_arr_delay
0,830.0,819,11.0,UA,1545,EWR,IAH,1400,2013-01-01 05:00:00,2013-01-01,869.92,1,11.0
1,850.0,830,20.0,UA,1714,LGA,IAH,1416,2013-01-01 05:00:00,2013-01-01,879.86,1,31.0
5,740.0,728,12.0,UA,1696,EWR,ORD,719,2013-01-01 05:00:00,2013-01-01,446.77,1,43.0
12,924.0,917,7.0,UA,194,JFK,LAX,2475,2013-01-01 06:00:00,2013-01-01,1537.89,1,50.0
13,923.0,937,-14.0,UA,1124,EWR,SFO,2565,2013-01-01 06:00:00,2013-01-01,1593.81,1,36.0
16,854.0,902,-8.0,UA,1187,EWR,LAS,2227,2013-01-01 06:00:00,2013-01-01,1383.79,1,28.0
24,858.0,915,-17.0,UA,1077,EWR,MIA,1085,2013-01-01 06:00:00,2013-01-01,674.19,1,11.0
26,945.0,931,14.0,UA,303,JFK,SFO,2586,2013-01-01 06:00:00,2013-01-01,1606.86,1,25.0
32,933.0,932,1.0,UA,496,LGA,IAH,1416,2013-01-01 06:00:00,2013-01-01,879.86,1,26.0
37,1016.0,947,29.0,UA,1665,EWR,LAX,2454,2013-01-01 06:00:00,2013-01-01,1524.84,1,55.0


In [34]:
df['origin'].replace({'EWR':'ewr', 'JFK':'jfk', 'LGA':'lga'}, inplace=True)
df.loc[df.origin=='ewr', ['dest']] ='BAY'

In [35]:
df.groupby('origin')['dest'].nunique().to_frame()

,dest
origin,
ewr,1
jfk,70
lga,68


In [37]:
df1 = df[~np.isnan(df['arr_delay'])]
df2 = df.loc[:,~df.columns.duplicated()]
df3 = df.drop_duplicates(subset=['origin'], keep='first')
df4 = pd.concat([df, pd.get_dummies(df['origin'])], axis=1)
df5 = pd.concat([df, pd.get_dummies(df.select_dtypes(include='object'))], axis=1)

In [41]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')